In [13]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

## Obtencion de casos por area de influencia INT

In [15]:
# Lee los casos de area de influencia
casos_area_de_influencia = build_features.leer_casos_area_de_influencia(COLUMNAS_POBLACION_INTERES)

## Obtencion de porcentajes de Hemodinamia

In [16]:
# Define la ruta donde estan los porcentajes de Hemodinamia
RUTA_PLANILLA_PORCENTAJES = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_INT.xlsx"
)

# Solamente lee los diagnosticos y el porcentaje de Hemodinamia
porcentaje_hmd = pd.read_excel(
    RUTA_PLANILLA_PORCENTAJES,
    sheet_name="porcentaje_trazadoras",
    usecols=["Diagnostico", "Porcentaje Pacientes Hemodinamia"],
)

# Deja los diagnosticos que SI aportan a la cantidad de pabellones de HMD (porcentaje > 0)
porcentaje_hmd = porcentaje_hmd.query("`Porcentaje Pacientes Hemodinamia` > 0")

# Formatea la columna de diagnosticos
porcentaje_hmd["Diagnostico"] = porcentaje_hmd["Diagnostico"].str.split(" - ").str[0]

# Deja los diagnosticos como el indice
porcentaje_hmd = porcentaje_hmd.set_index("Diagnostico")["Porcentaje Pacientes Hemodinamia"]

In [17]:
# Filtra los casos de area de influencia solamente a los diagnosticos que tengan un porcentaje de
# atencion cerrada mayor a 0
casos_area_de_influencia_para_ac = casos_area_de_influencia[
    casos_area_de_influencia.index.isin(porcentaje_hmd.index)
]

In [18]:
casos_area_de_influencia_para_ac

,Diagnosticos Contenidos,Macroproceso,Estadística,Casos (Cada 100.000),Edad Incidencia,Área de Influencia Formal,Área de Influencia Propuesta,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
Diagnostico,,,,,,,,,,,,,,,,,,,,,,,,,,
Enfermedad Coronaria,NaN,Hemodinamia,Incidencia,123.5,todos,NaN,Pais,18300.205896,18630.272833,18983.785331,19332.611299,19551.242779,19700.472502,19831.94369,19956.62105,20076.418142,20191.523737,20301.790791,20406.98185,20506.896216,20601.341145,20691.860596,20777.186814,20857.21945,20931.898892,21001.221165
